In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

data = pd.read_csv('/content/places_v6.csv')


encoder = OneHotEncoder(sparse=False)
categories_encoded = encoder.fit_transform(data[['categoriess']])


y = data['rating'].values


y_binned = pd.cut(y, bins=5, labels=False)
y_categorical = to_categorical(y_binned)


X_train, X_test, y_train, y_test = train_test_split(categories_encoded, y_categorical, test_size=0.2, random_state=42)


model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(5, activation='softmax'))


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


model.fit(X_train, y_train, epochs=250, batch_size=10, validation_split=0.2)


def predict_top_places(input_categories):

    input_data = np.zeros((1, categories_encoded.shape[1]))
    for category in input_categories:
        if category in encoder.categories_[0]:
            index = np.where(encoder.categories_[0] == category)[0][0]
            input_data[0, index] = 1


    predictions = model.predict(input_data)

    top_indices = predictions[0].argsort()[-5:][::-1]


    top_places = data.iloc[top_indices]


    for index, row in top_places.iterrows():
        print(f"Place: {row['name']}, Rating: {row['rating']}, Address: {row['formatted_address']}")


input_categories = ['hiking', 'waterfalls']
predict_top_places(input_categories)

Epoch 1/250


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - accuracy: 0.5127 - loss: 1.5559 - val_accuracy: 0.7344 - val_loss: 1.4131
Epoch 2/250
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6739 - loss: 1.3548 - val_accuracy: 0.7344 - val_loss: 1.1128
Epoch 3/250
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7040 - loss: 1.0189 - val_accuracy: 0.7344 - val_loss: 0.9128
Epoch 4/250
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7250 - loss: 0.8056 - val_accuracy: 0.7344 - val_loss: 0.8777
Epoch 5/250
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7362 - loss: 0.7175 - val_accuracy: 0.7344 - val_loss: 0.8678
Epoch 6/250
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7405 - loss: 0.6565 - val_accuracy: 0.7344 - val_loss: 0.8687
Epoch 7/250
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6974 - loss: 0.6881 - val_accuracy: 0.7344 - val_loss: 0.8813
Epoch 8/250
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7744 - loss: 0.5465 - val_accuracy: 0.7344 - val_loss: 0.

In [ ]:
model.save('model.h5')


In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.preprocessing import OneHotEncoder


model = load_model('/content/model.h5')


data = pd.read_csv('/content/places_v6.csv')


input_categories = ['hiking']


encoder = OneHotEncoder()
encoded_categories = encoder.fit_transform(data[['categoriess']]).toarray()


input_data = np.zeros((1, encoded_categories.shape[1]))
for category in input_categories:
    if category in encoder.categories_[0]:
        index = np.where(encoder.categories_[0] == category)[0][0]
        input_data[0, index] = 1


predictions = model.predict(input_data)


top_indices = predictions[0].argsort()[-5:][::-1]


top_places = data.iloc[top_indices]

for index, row in top_places.iterrows():
    print(f"Place: {row['name']}, Rating: {row['rating']}, Address: {row['formatted_address']}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step
Place: Alahana Pirivena, Rating: 4.8, Address: Polonnaruwa Sri Lanka
Place: Ahangama, Rating: 3.1449999999999982, Address: Ahangama Sri Lanka
Place: Ahungalla, Rating: 2.8339999999999974, Address: Ahungalla Sri Lanka
Place: Aanda Ella Fall, Rating: 4.2, Address: RatnapuraWewelwatte Rd Sri Lanka
Place: Aberdeen Waterfall, Rating: 4.8, Address: Ginigathhena Sri Lanka
